<a href="https://colab.research.google.com/github/JonNData/used-car-predictor/blob/master/notebooks/6_3_cl_price_modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import requests

In [2]:
pip install category_encoders

     |████████████████████████████████| 81kB 5.0MB/s 


In [3]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

from category_encoders.target_encoder import TargetEncoder
from sklearn.preprocessing import RobustScaler
# explicitly require this experimental feature
from sklearn.experimental import enable_iterative_imputer  # noqa
# now you can import normally from sklearn.impute
from sklearn.impute import IterativeImputer
from sklearn.linear_model import BayesianRidge

from sklearn.linear_model import RidgeCV
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb

from sklearn.model_selection import RandomizedSearchCV

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
# !pip install -U -q PyDrive
 
# from pydrive.auth import GoogleAuth
# from pydrive.drive import GoogleDrive
# from google.colab import auth
# from oauth2client.client import GoogleCredentials
 
# 1. Authenticate and create the PyDrive client.
# auth.authenticate_user()
# gauth = GoogleAuth()
# gauth.credentials = GoogleCredentials.get_application_default()
# drive = GoogleDrive(gauth)

# Go to link, sign in, and paste in result

In [0]:
# file_list = drive.ListFile({'q': "'1kE03N5LwmWDDsDescCPmlQxGsY7ZKEnT' in parents and trashed=false"}).GetList()
# for file1 in file_list:
#   print('title: %s, id: %s' % (file1['title'], file1['id']))

In [0]:
# downloaded = drive.CreateFile({'id':"1q_L-LMSE29jp64wP32VvJ8brX23L88NM"})  
# downloaded.GetContentFile('craigslist_cleaned_df_img.csv')    

In [0]:
# downloaded = drive.CreateFile({'id':"1BKZfd5WBLkryOSWI6t4g50bloTD8P6f3"})  
# downloaded.GetContentFile('vehicles_cl.csv')       

In [0]:
# downloaded = drive.CreateFile({'id':"12aa_5F_dECaFGjQR5KYZwToAqhHouUNi"})  
# downloaded.GetContentFile('vehicles.csv')       

In [5]:
df_orig = pd.read_csv('df_std_clean.csv')
df_orig

,Unnamed: 0,price,year,manufacturer,model,image_url,odometer,state,condition
0,0,10299.0,2012.0,acura,tl,https://images.craigslist.org/01414_3LIXs9EO33...,90186.0,nc,NaN
1,2,9500.0,2011.0,jaguar,xf,https://images.craigslist.org/00505_f22HGItCRp...,85000.0,nc,excellent
2,3,3995.0,2004.0,honda,element,https://images.craigslist.org/00E0E_eAUnhFF86M...,212526.0,nc,NaN
3,4,41988.0,2016.0,chevrolet,silverado k2500hd,https://images.craigslist.org/00S0S_8msT7RQquO...,NaN,ne,NaN
4,5,12995.0,2015.0,kia,sportage,https://images.craigslist.org/00000_2qdQpZA2hK...,85127.0,nc,NaN
...,...,...,...,...,...,...,...,...,...
137307,177899,7900.0,2013.0,chevrolet,captiva sport,https://images.craigslist.org/00N0N_laAdxuJXfp...,101063.0,fl,NaN
137308,177900,10495.0,2013.0,honda,civic,https://images.craigslist.org/00S0S_1GKwIrYHFL...,91379.0,fl,NaN
137309,177901,22995.0,2016.0,chevrolet,silverado 2500hd,https://images.craigslist.org/00h0h_cDLqh1fibf...,73972.0,fl,NaN
137310,177902,26995.0,2014.0,ram,2500,https://images.craigslist.org/00z0z_bwS9zBI3XJ...,83897.0,fl,NaN


In [6]:
df = df_orig.copy()
df.rename(columns={"Unnamed: 0": 'original_index'}, inplace=True)
df.describe()

,original_index,price,year,odometer
count,137312.000000,137312.000000,137312.000000,1.171360e+05
mean,89149.656862,14467.344893,2011.547694,9.593720e+04
std,51190.325889,10778.337111,4.880193,8.994037e+04
min,0.000000,500.000000,2000.000000,0.000000e+00
25%,44822.750000,6100.000000,2008.000000,4.495900e+04
50%,89078.500000,12000.000000,2012.000000,9.001200e+04
75%,133583.250000,19950.000000,2016.000000,1.330070e+05
max,177903.000000,189899.000000,2021.000000,1.000000e+07


In [0]:
df.dtypes

original_index      int64
price             float64
year              float64
manufacturer       object
model              object
image_url          object
odometer          float64
state              object
condition          object
dtype: object

In [0]:
df.isnull().sum()

original_index        0
price                 0
year                  0
manufacturer          0
model                 0
image_url             0
odometer          16144
state                 1
condition         45485
dtype: int64

## Split and define features + target 
3 features

In [0]:
train, test = train_test_split(df, test_size=0.15, random_state=11)

X_features = ['odometer', 'year', 'model', 'manufacturer']
target = 'price'

X_train = train[X_features]
y_train = train[target]

X_test = test[X_features]
y_test = test[target]

## Preprocessing

In [0]:
te = TargetEncoder()
X_train_encoded = te.fit_transform(X_train, y_train)
X_test_encoded = te.transform(X_test)

imputer = IterativeImputer(BayesianRidge())
X_train_imputed = imputer.fit_transform(X_train_encoded, y_train)
X_test_imputed = imputer.transform(X_test_encoded)

rs = RobustScaler()
X_train_scaled = rs.fit_transform(X_train_imputed)
X_test_scaled = rs.transform(X_test_imputed)

In [0]:
from scipy import stats
stats.describe(X_train_scaled)

DescribeResult(nobs=351477, minmax=(array([-1.15722063, -1.71428571, -1.26206598, -2.25278885]), array([752.77146879,   1.28571429,  18.12758377,  17.24937853])), mean=array([ 0.04563957, -0.09682121,  0.15220634, -0.24481055]), variance=array([4.29677758, 0.48418033, 0.65187965, 0.54845511]), skewness=array([277.01138551,  -0.38270708,   1.85177752,   1.36767677]), kurtosis=array([ 9.91366280e+04, -7.89733748e-01,  1.27720730e+01,  2.13348995e+01]))

In [0]:
df['odometer'].describe()

count    3.505170e+05
mean     1.002343e+05
std      1.813931e+05
min      0.000000e+00
25%      4.860000e+04
50%      9.460800e+04
75%      1.379430e+05
max      6.480922e+07
Name: odometer, dtype: float64

## Ridge Regression

In [0]:
rcv = RidgeCV()
rcv.fit(X_train_scaled, y_train)
rcv.score(X_train_scaled, y_train)

In [0]:
y_pred_ridge = rcv.predict(X_test_scaled)
mean_absolute_error(y_test, y_pred_ridge)

## Random Forest

In [0]:
rf = RandomForestRegressor()
rf.fit(X_train_scaled, y_train)
y_pred_rf = rf.predict(X_test_scaled)

mean_absolute_error(y_test, y_pred_rf)

1760.547211313215

In [0]:
rf.feature_importances_

array([0.12774785, 0.28118137, 0.54667324, 0.04439754])

In [0]:
from sklearn.pipeline import make_pipeline

pipe = make_pipeline(
    TargetEncoder(),
    IterativeImputer(BayesianRidge()),
    RobustScaler(),
    RandomForestRegressor(n_estimators=5, max_depth=20)
)
pipe.fit(X_train, y_train)


Pipeline(memory=None,
         steps=[('targetencoder',
                 TargetEncoder(cols=['model', 'manufacturer'],
                               drop_invariant=False, handle_missing='value',
                               handle_unknown='value', min_samples_leaf=1,
                               return_df=True, smoothing=1.0, verbose=0)),
                ('iterativeimputer',
                 IterativeImputer(add_indicator=False,
                                  estimator=BayesianRidge(alpha_1=1e-06,
                                                          alpha_2=1e-06,
                                                          alpha_init=None,
                                                          compute_s...
                 RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,
                                       criterion='mse', max_depth=20,
                                       max_features='auto', max_leaf_nodes=None,
                                       max_sample

In [0]:
y_pred_pipe = pipe.predict(X_test)
mean_absolute_error(y_test, y_pred_pipe)


2098.266357643709

In [0]:
from joblib import load, dump

dump(pipe, 'targetiterrobustforest.joblib', compress=True)


['targetiterrobustforest.joblib']

In [0]:
# Tune it?

# Create the random grid
random_grid = {'n_estimators': [100, 150],
               'max_depth': [25, 40],
               'min_samples_split': [2, 5, 10],
               'min_samples_leaf': [1, 2, 4],
               'bootstrap': [True, False]}
rf1 = RandomForestRegressor()
rf_random = RandomizedSearchCV(
            estimator = rf1, param_distributions = random_grid, 
            n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)

# Fit the random search model
rf_random.fit(X_train_scaled, y_train)


Fitting 3 folds for each of 72 candidates, totalling 216 fits


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:281: UserWarning: The total space of parameters 72 is smaller than n_iter=100. Running 72 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  9.2min
/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed: 44.4min
[Parallel(n_jobs=-1)]: Done 216 out of 216 | elapsed: 65.6min finished


RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=RandomForestRegressor(bootstrap=True,
                                                   ccp_alpha=0.0,
                                                   criterion='mse',
                                                   max_depth=None,
                                                   max_features='auto',
                                                   max_leaf_nodes=None,
                                                   max_samples=None,
                                                   min_impurity_decrease=0.0,
                                                   min_impurity_split=None,
                                                   min_samples_leaf=1,
                                                   min_samples_split=2,
                                                   min_weight_fraction_leaf=0.0,
                                                   n_estimators=100,
                              

In [0]:
rf_random.best_score_

0.8195643283673594

In [0]:
rf_random.best_params_

{'bootstrap': True,
 'max_depth': 25,
 'min_samples_leaf': 2,
 'min_samples_split': 2,
 'n_estimators': 100}

In [0]:
y_pred_search = rf_random.predict(X_test_scaled)
mean_absolute_error(y_test, y_pred_search)

2456.4202428938074

## XGBoost


In [0]:
#%% split training set to validation set 
data_tr  = xgb.DMatrix(X_train_scaled, label=y_train)
data_cv  = xgb.DMatrix(X_test_scaled   , label=y_test)
evallist = [(data_tr, 'train'), (data_cv, 'valid')]

In [0]:
parms = {'max_depth':25, #maximum depth of a tree
         'objective':'reg:squarederror',
         'eval_metric': 'mae',
         'eta'      :0.1,
         'subsample':0.8,#SGD will use this percentage of data
         'lambda '  :1, #L2 regularization term,>1 more conservative 
         'colsample_bytree ':0.9,
         'colsample_bylevel':1,
         'min_child_weight': 5}
modelx = xgb.train(parms, data_tr, num_boost_round=500, evals = evallist,
                  early_stopping_rounds=10, maximize=False, 
                  verbose_eval=10)

[0]	train-mae:12722.1	valid-mae:12687.8
Multiple eval metrics have been passed: 'valid-mae' will be used for early stopping.

Will train until valid-mae hasn't improved in 10 rounds.
[10]	train-mae:4810.53	valid-mae:4891.95
[20]	train-mae:2317.47	valid-mae:2653.34
[30]	train-mae:1539.05	valid-mae:2075.41
[40]	train-mae:1279.74	valid-mae:1916.57
[50]	train-mae:1186.35	valid-mae:1869.82
[60]	train-mae:1145.15	valid-mae:1852.51
[70]	train-mae:1118.02	valid-mae:1841.06
[80]	train-mae:1096.06	valid-mae:1831.71
[90]	train-mae:1075.76	valid-mae:1824.01
[100]	train-mae:1061.19	valid-mae:1818.27
[110]	train-mae:1035.15	valid-mae:1808.98
[120]	train-mae:1017.32	valid-mae:1802.37
[130]	train-mae:999.455	valid-mae:1796.2
[140]	train-mae:980.111	valid-mae:1789.05
[150]	train-mae:962.014	valid-mae:1782.5
[160]	train-mae:942.437	valid-mae:1775.94
[170]	train-mae:925.171	valid-mae:1769.8
[180]	train-mae:906.683	valid-mae:1763.5
[190]	train-mae:886.114	valid-mae:1756.79
[200]	train-mae:865.711	valid-ma

In [0]:
# [999]	train-mae:1697.07	valid-mae:2067.33 max_depth 8
# [499]	train-mae:1034.21	valid-mae:1844.92 max_depth 15
# [499]	train-mae:1511.85	valid-mae:1967.65 lambda 2 eta 0.1(same exact for lambda1) 
# [499]	train-mae:787.267	valid-mae:1735.56 max_depth 25
# [499]	train-mae:456.659	valid-mae:1653.16 min_child_weight 5

## Fuzzy Wuzzy attemps

In [0]:
pip install fuzzywuzzy

In [0]:
from fuzzywuzzy import fuzz, process

/usr/local/lib/python3.6/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [0]:
df_epa = pd.read_csv('vehicles.csv')

In [0]:
# Start with epa, compare models where year and make is the same

# for row in df_epa:
#   if df_epa.year == df_cl.year:
#     print(fuzz.ratio(df_epa['model'][row], df_cl['model'][row]))



In [0]:
df_cl = df

In [0]:
# Lets look at a simplified list

epa_models = sorted(df_epa['model'].unique())
cl_models = sorted(df_cl.model.unique())


for model in epa_models:
  for x  in range(len(cl_models)):
    print(fuzz.ratio(model, cl_models[x]))

In [0]:
epa_models[2888]

In [0]:
grand_cherokee_ratios = []
for model in cl_models:
  grand_cherokee_ratios.append(fuzz.ratio(epa_models[2888], model))
max(grand_cherokee_ratios)

In [0]:
grand_cherokee_ratios.index(84)

In [0]:
cl_models[18845]

In [0]:
df_epa['model'].nunique(), df['model'].nunique()

In [0]:
# Input Make model Year get a prediction
# sample_model = 'sivics'

cl_models = sorted(df_cl.model.unique())

def match_models(sample_model):
  model_ratios = []
  for model in cl_models:
    model_ratios.append(fuzz.ratio(sample_model, model))
  max_match = max(model_ratios)
  index_of_match = model_ratios.index(max_match)
  return cl_models[index_of_match]

In [0]:
match_models('Wrangler Rubicon 4wd')

In [0]:
def match_models(sample_model):
  model_ratios = []
  for model in cl_models:
    model_ratios.append(fuzz.ratio(sample_model, model))
  max_match = max(model_ratios)
  index_of_match = model_ratios.index(max_match)
  return cl_models[index_of_match]

### Refactor to us the process.extract fuzzy

In [0]:
df_cl = df
cl_models = sorted(df_cl.model.unique())

In [0]:
%time
process.extractOne('civic 4wd', cl_models, scorer=fuzz.token_sort_ratio)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.53 µs


('civic lx 4dr', 76)

In [0]:
%time
match_models('civic 4wd')

CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 9.06 µs


'civic dx'

In [0]:
%time
process.extract('z3 coupe', cl_models, scorer=fuzz.partial_ratio)

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 6.44 µs


[('3', 100),
 ('z3', 100),
 ('370z coupe', 93),
 ('370z coupe 2d', 88),
 ('cr-z coupe 2d', 88)]

In [0]:
%time
match_models('z3 coupe')

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.48 µs


'm3 coupe'

In [0]:
%time
process.extract('cx-9 4wd', cl_models, scorer=fuzz.token_sort_ratio)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.01 µs


[('crv ex 4wd', 67),
 ('cx-9', 67),
 ('crv exl 4wd', 63),
 ('cr-v 4wd', 62),
 ('cx-5 awd', 62)]

In [0]:
%time
match_models('cx-9 4wd')

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.01 µs


'cr-v 4wd'

In [0]:
%time
process.extract('range rover evoque cabriolet', cl_models, scorer=fuzz.token_set_ratio)

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 7.87 µs


[('cabriolet', 100),
 ('evoque', 100),
 ('evoque d', 86),
 ('s5 cabriolet', 86),
 ('clk350 cabriolet', 72)]

In [0]:
%time
match_models('range rover evoque cabriolet')

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 5.01 µs


'murano crosscabriolet'

In [0]:
%time
process.extract('e500 4matic (wagon)', cl_models, scorer=fuzz.token_sort_ratio)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.01 µs


[('e350 4matic', 71),
 ('e350 club wagon', 69),
 ('benz e500 4matic', 67),
 ('benz s500 4matic', 67),
 ('c300 4matic sport', 65)]

In [0]:
%time
match_models('e500 4matic (wagon)')

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 7.87 µs


'e350 4matic'

### Fuzzy makes


In [0]:
# Try to get rid of rare makes S and S represent

cl_makes = sorted(df_cl.manufacturer.unique())

def match_makes(sample_make):
  make_ratios = []
  for make in cl_makes:
    make_sublist.append(fuzz.ratio(sample_make, make))

  max_match = max(make_ratios)
  index_of_match = make_ratios.index(max_match)
  return cl_makes[index_of_match]

In [0]:
df_cl.shape

In [0]:
match_makes('Aston Martin')

## Image Retrieval

In [0]:
df_cl_orig = pd.read_csv('vehicles_cl.csv')
df_cl_orig.head()

,Unnamed: 0,year,manufacturer,model,image_url
0,0,2012.0,acura,tl,https://images.craigslist.org/01414_3LIXs9EO33...
1,2,2011.0,jaguar,xf,https://images.craigslist.org/00505_f22HGItCRp...
2,3,2004.0,honda,element,https://images.craigslist.org/00E0E_eAUnhFF86M...
3,4,2016.0,chevrolet,silverado k2500hd,https://images.craigslist.org/00S0S_8msT7RQquO...
4,5,2015.0,kia,sportage,https://images.craigslist.org/00000_2qdQpZA2hK...


In [0]:
df_img = df_cl_orig.copy()
df_img = df_img[['price', 'year', 'manufacturer', 'model', 'image_url']]
df_img.head()

In [0]:
## Cleaning

df_img['price'] = df_img['price'].replace(0, np.NaN)
df_img = df_img.dropna(subset=['price'])
df_img = df_img[(df_img['price'] >= np.percentile(df_img['price'], 1)) &
          (df_img['price'] <= np.percentile(df_img['price'], 99))]
df_img = df_img.dropna(subset=['manufacturer', 'model', 'year'])

df_img[df_img['price']<500]
df_img = df_img.groupby("model").filter(lambda x: len(x) > 2)


In [0]:
df_img = df_img[df_img.year >= 2000]
df_img.shape

In [0]:
df_img = df_img[['year', 'manufacturer', 'model', 'image_url']]
df_img

In [0]:
# df_img.to_csv('craigslist_cleaned_df_img')

In [0]:
def match_models(sample_model):
    '''
    This function takes in a given model from the EPA dataset and
    uses the Fuzzy Wuzzy library to match the input string to the closest
    string in the Craigslist dataset.
    '''
    model_ratios = []
    for car in cl_models:
        model_ratios.append(fuzz.ratio(sample_model, car))
    max_match = max(model_ratios)
    index_of_match = model_ratios.index(max_match)
    return cl_models[index_of_match]


In [0]:
# Call the function to get the CL equivalent car
model_fz = match_models(model_lower)

In [0]:
def fetch_img(car_model, year):
  """
  Get from sample input car to return the url. If none found,
   check next and previous year. If none available, give none found image
  """
  df_models = df_img[df_img['model'] == car_model]
  df_models_at_year = df_models[df_models['year'] == year ]
  index_of_model_year = df_models_at_year.index[0:10]

  list_urls = list(df_img['image_url'][index_of_model_year])

  if len(index_of_model_year) < 1:
    df_models_at_year = df_models[df_models['year'] == (year + 1)]
    index_of_model_year = df_models_at_year.index[0:10]
    list_urls = list(df_img['image_url'][index_of_model_year])

    if len(list_urls) == 0:
      df_models_at_year = df_models[df_models['year'] == (year - 1)]
      index_of_model_year = df_models_at_year.index[0:10]
      list_urls = list(df_img['image_url'][index_of_model_year])
      #print('No cars in specified year, trying the previous year')  

      if len(list_urls) == 0:
        return ['https://raw.githubusercontent.com/Lambda-School-Labs/street-smarts-ds/master/data/noImage_large.png']

      return list_urls  

    #print('No cars in specified year, trying the next year')
    return list_urls

  return list_urls

In [0]:
output = fetch_img('model 3', 2020)

In [0]:
output

In [0]:
df_img.shape

In [0]:
cl_img_models = df_img.model.unique()
len(cl_img_models)

## Cleaning out broken links

In [0]:
df_img_orig = pd.read_csv('craigslist_cleaned_df_img.csv')
df_img = df_img_orig.copy()

In [0]:
# Test the function as is

output = fetch_img('model 3', 2020)

In [0]:
def test_get_list_urls_check_status_code_equals_200(url):
  response = requests.get(url)
  if response.status_code == 200:
    url = url
  else:
    return np.NaN
  return url

In [0]:
for img in output:
  img = test_get_list_urls_check_status_code_equals_200(img)
  print(img)

https://images.craigslist.org/00h0h_gb13SJamasm_600x450.jpg
nan
nan
https://images.craigslist.org/00606_a2AR614CprU_600x450.jpg
https://images.craigslist.org/00s0s_lP4xMk7aH1A_600x450.jpg
https://images.craigslist.org/00000_g1ckhuAfeiG_600x450.jpg
https://images.craigslist.org/00P0P_joAX9VGHRou_600x450.jpg
https://images.craigslist.org/00606_a2AR614CprU_600x450.jpg
https://images.craigslist.org/00f0f_iCGas2Rnor1_600x450.jpg
https://images.craigslist.org/00101_gOO35IlSFWf_600x450.jpg


In [0]:
# Now refactor our function to include Status code checking

In [0]:
# Filter our data frame so that the column only has currently working urls
df_img.drop(columns="Unnamed: 0", inplace=True)
print(df_img.shape)
df_img.head()

(413414, 4)


,year,manufacturer,model,image_url
0,2012.0,acura,tl,https://images.craigslist.org/01414_3LIXs9EO33...
1,2011.0,jaguar,xf,https://images.craigslist.org/00505_f22HGItCRp...
2,2004.0,honda,element,https://images.craigslist.org/00E0E_eAUnhFF86M...
3,2016.0,chevrolet,silverado k2500hd,https://images.craigslist.org/00S0S_8msT7RQquO...
4,2015.0,kia,sportage,https://images.craigslist.org/00000_2qdQpZA2hK...


In [0]:
df_sample = df_img.sample(200, random_state=11)
df_sample

,year,manufacturer,model,image_url
369182,2011.0,honda,pilot ex-l,https://images.craigslist.org/00B0B_kHP5NJHnGG...
41517,2016.0,jeep,cherokee trailhawk,https://images.craigslist.org/00l0l_gQKakrucRl...
146485,2007.0,toyota,yaris,https://images.craigslist.org/00x0x_dFa66HKlBo...
329707,2001.0,honda,civic,https://images.craigslist.org/00F0F_3FWRN7v5Va...
74819,2008.0,ford,f150 xlt,https://images.craigslist.org/01414_8o6THP6YXp...
...,...,...,...,...
215705,2012.0,dodge,charger,https://images.craigslist.org/00g0g_l1kbcOksji...
237402,2002.0,volkswagen,jetta,https://images.craigslist.org/00G0G_4gGHjhvuv9...
189256,2008.0,chevrolet,silverado,https://images.craigslist.org/00q0q_iDYjduZfFM...
139635,2011.0,gmc,terrain slt-1,https://images.craigslist.org/00Y0Y_25YGIRNOaz...


In [0]:
%%time
df_sample['image_url'] = df_sample['image_url'].apply(test_get_list_urls_check_status_code_equals_200)

CPU times: user 2.4 s, sys: 145 ms, total: 2.54 s
Wall time: 25.1 s


In [0]:
df_sample

,Unnamed: 0,year,manufacturer,model,image_url
369182,482903,2011.0,honda,pilot ex-l,https://images.craigslist.org/00B0B_kHP5NJHnGG...
41517,54394,2016.0,jeep,cherokee trailhawk,NaN
146485,189909,2007.0,toyota,yaris,https://images.craigslist.org/00x0x_dFa66HKlBo...
329707,430445,2001.0,honda,civic,https://images.craigslist.org/00F0F_3FWRN7v5Va...
74819,97034,2008.0,ford,f150 xlt,https://images.craigslist.org/01414_8o6THP6YXp...
...,...,...,...,...,...
215705,278769,2012.0,dodge,charger,https://images.craigslist.org/00g0g_l1kbcOksji...
237402,307336,2002.0,volkswagen,jetta,https://images.craigslist.org/00G0G_4gGHjhvuv9...
189256,245297,2008.0,chevrolet,silverado,https://images.craigslist.org/00q0q_iDYjduZfFM...
139635,180890,2011.0,gmc,terrain slt-1,https://images.craigslist.org/00Y0Y_25YGIRNOaz...


In [0]:
df_sample['image_url'].isnull().sum()

49

In [0]:
# Logic for dealing with URL nans
def fetch_img_NaN(car_model, year):
  """
  Get from sample input car to return the url. If none found,
   check next and previous year. If none available, give none found image
  """
  df_models = df_sample[df_sample['model'] == car_model]
  df_models_at_year = df_models[df_models['year'] == year ]
  index_of_model_year = df_models_at_year.index[0:10]

  # Check for nans
  list_urls = list(df_sample['image_url'][index_of_model_year])

  clean_list_urls = [x for x in list_urls if x is not np.NaN]

  if len(clean_list_urls) < 1:
    df_models_at_year = df_models[df_models['year'] == (year + 1)]
    index_of_model_year = df_models_at_year.index[0:10]
    list_urls = list(df_sample['image_url'][index_of_model_year])
    clean_list_urls = [x for x in list_urls if x is not np.NaN]

    if len(clean_list_urls) == 0:
      df_models_at_year = df_models[df_models['year'] == (year - 1)]
      index_of_model_year = df_models_at_year.index[0:10]
      list_urls = list(df_sample['image_url'][index_of_model_year])
      clean_list_urls = [x for x in list_urls if x is not np.NaN]
      #print('No cars in specified year, trying the previous year')  

      if len(clean_list_urls) == 0:
        return ['https://raw.githubusercontent.com/Lambda-School-Labs/street-smarts-ds/master/data/noImage_large.png']

      return clean_list_urls  

    #print('No cars in specified year, trying the next year')
    return clean_list_urls

  return clean_list_urls

In [0]:
fetch_img_NaN('rx 350', 2007)

['https://raw.githubusercontent.com/Lambda-School-Labs/street-smarts-ds/master/data/noImage_large.png']

In [0]:
def get_valid_urls_from_model(car_model, yearx):
    """
    Take the df with models of interest and return list of non-nan urls
    """
    df_models = df_sample[df_sample['model'] == car_model]
    df_models_at_year = df_models[df_models['year'] == yearx]
    index_of_model_year = df_models_at_year.index[0:10]
    list_urls = list(df_sample['image_url'][index_of_model_year])
    clean_list_urls = [x for x in list_urls if x is not np.NaN]
    return clean_list_urls

In [0]:
  # Attempt refactor
def status_200_or_nan(url):
    response = requests.get(url)
    if response.status_code == 200:
      url = url
    else:
      return np.NaN
    return url
def year_to_urls(car_model, year):
    df_models = df_cl[df_cl['model'] == car_model]
    df_models_at_year = df_models[df_models['year'] == year ]
    index_of_model_year = df_models_at_year.index[0:4]

    list_urls = list(df_cl['image_url'][index_of_model_year])

    list_nan_urls = [status_200_or_nan(x) for x in list_urls]
    clean_list_urls = [x for x in list_nan_urls if x is not np.NaN]

def fetch_img(car_model, year):
  year_to_urls(car_model, year)

  # Check if there are any cars in the current year, else check next year
  if len(clean_list_urls) == 0:
    year = year + 1
    year_to_urls(car_model, year)
      
    if len(clean_list_urls) == 0:
      year = year - 2
      year_to_urls(car_model, year)

      if len(clean_list_urls) == 0:
          return ['https://raw.githubusercontent.com/Lambda-School-Labs/street-smarts-ds/master/data/noImage_large.png']
      return clean_list_urls  

      return clean_list_urls

  return clean_list_urls

IndentationError: ignored

In [0]:
def status_200_or_nan(url):
  response = requests.get(url)
  if response.status_code == 200:
    return url
  else:
    return np.NaN

def year_to_urls(car_model, year):
  df_models = df_cl[df_cl['model'] == car_model]
  df_models_at_year = df_models[df_models['year'] == year]
  index_of_model_year = df_models_at_year.index[0:4]

  list_urls = list(df_cl['image_url'][index_of_model_year])
  list_w_nan = [status_200_or_nan(x) for x in list_urls]
  clean_list_urls = [x for x in list_w_nan if x is not np.NaN]
  return clean_list_urls

def fetch_image(car_model, year):
  clean_list_urls = year_to_urls(car_model, year)

  #check for cars, else search next year
  if len(clean_list_urls) == 0:
    year1 = year + 1
    clean_list_urls = year_to_urls(car_model, year1)

    if len(clean_list_urls) == 0:
      year0 = year - 1
      clean_list_urls = year_to_urls(car_model, year0)

      if len(clean_list_urls) == 0:
        return ['https://raw.githubusercontent.com/Lambda-School-Labs/street-smarts-ds/master/data/noImage_large.png']
      return clean_list_urls
    return clean_list_urls
  return clean_list_urls

In [0]:
%time
fetch_image('civic 4wd', 2007)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.44 µs


['https://raw.githubusercontent.com/Lambda-School-Labs/street-smarts-ds/master/data/noImage_large.png']

In [0]:
df_models = df_sample[df_sample['model'] == 'escape']

get_valid_urls_from_model(2015)

['https://images.craigslist.org/00S0S_2LHJ89djgTr_600x450.jpg']

In [0]:
# df_img_cleaned = df_img.copy()
# df_img_cleaned['image_url'] = df_img_cleaned['image_url'].apply(test_get_list_urls_check_status_code_equals_200)

In [0]:
#df_img_cleaned.to_csv('cl_img_cleaned.csv')